## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-12-03-00-16 +0000,bbc,Federal judge says voice-over artists' AI laws...,https://www.bbc.com/news/articles/cedgzj8z1wjo
1,2025-07-12-02-59-22 +0000,bbc,Why cockpit audio deepens the mystery of Air I...,https://www.bbc.com/news/articles/cx2gy78gpnqo
2,2025-07-12-02-56-04 +0000,nypost,LA Mayor Karen Bass signs order to provide cas...,https://nypost.com/2025/07/11/us-news/la-mayor...
3,2025-07-12-02-13-21 +0000,bbc,Arizona wildfires and extreme heat prompt eva...,https://www.bbc.com/news/articles/czjk10wwmkeo
4,2025-07-12-02-08-29 +0000,nypost,Trump grants ICE ability to arrest ‘SLIMEBALL’...,https://nypost.com/2025/07/11/us-news/trump-gr...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2428/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
35,trump,21
129,canada,7
200,tariff,5
63,rate,4
303,war,4


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
37,2025-07-11-19-40-14 +0000,nyt,Trump’s Latest Canada Tariff Threats Come Desp...,https://www.nytimes.com/2025/07/11/world/canad...,44
25,2025-07-11-22-51-48 +0000,nypost,Dow drops nearly 300 points after Trump teases...,https://nypost.com/2025/07/11/us-news/dow-drop...,43
21,2025-07-11-23-03-00 +0000,wsj,"Stock Market News, July 11, 2025: Dow Finishes...",https://www.wsj.com/livecoverage/stock-market-...,41
53,2025-07-11-15-59-15 +0000,nypost,Head Start education program will be cut off f...,https://nypost.com/2025/07/11/us-news/head-sta...,41
57,2025-07-11-15-18-54 +0000,nypost,Trump teases ‘major statement’ on Russia after...,https://nypost.com/2025/07/11/us-news/trump-te...,41


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
37,44,2025-07-11-19-40-14 +0000,nyt,Trump’s Latest Canada Tariff Threats Come Desp...,https://www.nytimes.com/2025/07/11/world/canad...
61,23,2025-07-11-14-04-24 +0000,nypost,Sen. Bill Cassidy’s campaign touts record-sett...,https://nypost.com/2025/07/11/us-news/sen-bill...
6,20,2025-07-12-01-53-00 +0000,wsj,A dispute over the Federal Reserve’s renovatio...,https://www.wsj.com/economy/central-banking/je...
53,20,2025-07-11-15-59-15 +0000,nypost,Head Start education program will be cut off f...,https://nypost.com/2025/07/11/us-news/head-sta...
16,20,2025-07-11-23-48-26 +0000,nypost,Judge releases bizarre letter from would-be Tr...,https://nypost.com/2025/07/11/us-news/judge-re...
35,19,2025-07-11-20-31-41 +0000,nypost,Trump mourns ‘a lot of young angels’ after sur...,https://nypost.com/2025/07/11/us-news/texas-fl...
4,17,2025-07-12-02-08-29 +0000,nypost,Trump grants ICE ability to arrest ‘SLIMEBALL’...,https://nypost.com/2025/07/11/us-news/trump-gr...
59,17,2025-07-11-14-53-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...
3,14,2025-07-12-02-13-21 +0000,bbc,Arizona wildfires and extreme heat prompt eva...,https://www.bbc.com/news/articles/czjk10wwmkeo
45,14,2025-07-11-17-37-00 +0000,wsj,Nuclear power exemplifies how revamping dated ...,https://www.wsj.com/economy/trumps-unsung-econ...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
